In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.applications.efficientnet import preprocess_input

import os
import cv2
from os import walk
import glob as gb
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
SEED = 1000
IMG_SIZE = 224
BATCH_SIZE = 32

TRAIN_DIR = '../input/caltech-zipped/Caltech Bird 2/Train'
VALID_DIR = '../input/caltech-zipped/Caltech Bird 2/Validation'
TEST_DIR = '../input/caltech-zipped/Caltech Bird 2/Test'

In [4]:
from glob import glob
training_dir=TRAIN_DIR
validation_dir=VALID_DIR
image_files = glob(training_dir + '/*/*')
valid_image_files = glob(validation_dir + '/*/*')

In [5]:
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

In [6]:
from keras.models import Model
from keras.layers import Flatten, Dense
from tensorflow.keras.applications import EfficientNetB0

#IMG_SIZE=80

IMAGE_SIZE = [IMG_SIZE, IMG_SIZE] 


En = EfficientNetB0(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False) 


for layer in En.layers:
    layer.trainable = False

x = Flatten()(En.output)

x = Dense(num_classes, activation = 'softmax')(x)  

model = Model(inputs = En.input, outputs = x)

In [7]:
model.summary()

In [8]:
training_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=False,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory(
    training_dir, 
    target_size = IMAGE_SIZE, 
    batch_size = BATCH_SIZE, 
    class_mode = 'sparse')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size = IMAGE_SIZE, 
    batch_size = BATCH_SIZE, 
    class_mode = 'sparse')

In [9]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 40:
        lr *= 0.5e-3
    elif epoch > 30:
        lr *= 1e-3
    elif epoch > 20:
        lr *= 1e-2
    elif epoch > 10:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr
lr_scheduler = LearningRateScheduler(lr_schedule)

In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=lr_schedule(0)), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [11]:
h = model.fit(training_generator, 
              epochs=50,
              steps_per_epoch = 8190 // BATCH_SIZE,
              validation_data = validation_generator,
              validation_steps = 2311 // BATCH_SIZE,
              callbacks=lr_scheduler,
              verbose=2
             )

In [13]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(h.history['accuracy'],  label='train accuracy')
plt.plot(h.history['val_accuracy'],  label = 'validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend(loc='lower right')

plt.subplot(1,2,2)
plt.plot(h.history['loss'],  label='train loss')
plt.plot(h.history['val_loss'],  label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.ylim([0,10])
plt.legend(loc='upper right')

plt.savefig ("EfficientNet.pdf",dpi=300)

plt.show()